## Name: Shelmith Nyagathiri Kariuki
## Date: 1st July , 2018
## Description: Analytics Vidhya; Loan Prediction III Competition

In [1]:
#1. Loading the libraries that we will need.
import os #used to set the working directory
import numpy as np #used for mathematical computations
import pandas as pd #used for manipulation of dataframes
import matplotlib.pyplot as plt #used for basic plots 
import seaborn as sns #used for producing pretty plots
import sklearn #used for pre processing data and fitting machine learning models.
import statsmodels.api as sm

In [83]:
#2. Setting the working directory
os.chdir("/Users/busara/Documents/Personal Development/Python/Datasets")
os.getcwd()

'/Users/busara/Documents/Personal Development/Python/Datasets'

In [84]:
#3. Reading in the loan datasets
train_df = pd.read_csv("LoanPredictionIII/train_u6lujuX_CVtuZ9i (1).csv")
del train_df["Loan_ID"]
test_df = pd.read_csv("LoanPredictionIII/test_Y3wMUE5_7gLdaTN.csv")
Loan_ID1= test_df.Loan_ID
del test_df["Loan_ID"]


In [85]:
#4. Examine the number of rows and columns in each of the datasets
print("The train dataset has %d rows and %d columns" %(train_df.shape[0],train_df.shape[1]))
print("The test dataset had %d rows and %d columns" %(test_df.shape[0],test_df.shape[1]))

The train dataset has 614 rows and 12 columns
The test dataset had 367 rows and 11 columns


In [86]:
#5. Combining both datasets for easier cleaning.
train_df["dataset"] ="train"
test_df["dataset"] = "test"
loan_data = train_df.append(test_df,sort=True)
print("The combined dataset had %d rows and %d columns" %(loan_data.shape[0],loan_data.shape[1]))
#loan_data

The combined dataset had 981 rows and 13 columns


In [87]:
#5. Examine missingness 
loan_data.info()
# Credit_History has 902 obs, 79 missing
# Dependents has 956 values,25 missing
# Gender has 957 values, 24 missing
# LoanAmount has 954 values, 27 missing
# Loan_Amount_Term has 961 values, 20 missing
# Married has 978 values, 3 missing
# Self_Employed has 926 values, 55 missing


<class 'pandas.core.frame.DataFrame'>
Int64Index: 981 entries, 0 to 366
Data columns (total 13 columns):
ApplicantIncome      981 non-null int64
CoapplicantIncome    981 non-null float64
Credit_History       902 non-null float64
Dependents           956 non-null object
Education            981 non-null object
Gender               957 non-null object
LoanAmount           954 non-null float64
Loan_Amount_Term     961 non-null float64
Loan_Status          614 non-null object
Married              978 non-null object
Property_Area        981 non-null object
Self_Employed        926 non-null object
dataset              981 non-null object
dtypes: float64(4), int64(1), object(8)
memory usage: 107.3+ KB


In [88]:
#6. Impute missing values. (I need to look for a better way)
for col in ['Credit_History','Dependents','Gender','LoanAmount','Loan_Amount_Term',
            'Married','Self_Employed','LoanAmount']:
    loan_data[col]=loan_data[col].fillna(method = 'bfill')

loan_data.info()


# from fancyimpute import MICE
# #loan_data2 = pd.DataFrame(data=MICE.complete(loan_data), columns=loan_data.columns, index=loan_data.index)
# MICE.complete?


<class 'pandas.core.frame.DataFrame'>
Int64Index: 981 entries, 0 to 366
Data columns (total 13 columns):
ApplicantIncome      981 non-null int64
CoapplicantIncome    981 non-null float64
Credit_History       981 non-null float64
Dependents           981 non-null object
Education            981 non-null object
Gender               981 non-null object
LoanAmount           981 non-null float64
Loan_Amount_Term     981 non-null float64
Loan_Status          614 non-null object
Married              981 non-null object
Property_Area        981 non-null object
Self_Employed        981 non-null object
dataset              981 non-null object
dtypes: float64(4), int64(1), object(8)
memory usage: 107.3+ KB


In [89]:
#6. Label Encoding all the categorical variables.
from sklearn.preprocessing import LabelEncoder
# for col in train_df.columns.values:
#     if train_df[col].dtypes == "O":
#         train_df[col] = train_df[col].astype('category')

labenc = LabelEncoder()
for col in loan_data.columns.values:
    if loan_data[col].dtypes == "object":
        loan_data[col] = labenc.fit_transform(loan_data[col].astype(str))

In [90]:
#7. Normalizing the continuous variables. (Giving me problems)
# from sklearn.preprocessing import MinMaxScaler
# for col in loan_data.columns.values:
#     if loan_data[col].dtypes =="float64":
#         loan_data[col].values = MinMaxScaler().fit_transform(loan_data[col].values)


In [91]:
#8. Scaling the continous variables.
from sklearn.preprocessing import scale
loan_data[['ApplicantIncome', 'CoapplicantIncome','LoanAmount','Loan_Amount_Term']] = pd.DataFrame(scale(loan_data[['ApplicantIncome', 'CoapplicantIncome',
                 'LoanAmount','Loan_Amount_Term']],axis=0))
#loan_data

In [92]:
#9. Separate the datasets back to training and testing data.

train_df2 = loan_data.loc[loan_data["dataset"]==1]
del train_df2["dataset"]
test_df2 = loan_data.loc[loan_data["dataset"]==0]
del test_df2["dataset"]
del test_df2["Loan_Status"]

print("The train dataset has %d rows and %d columns" %(train_df2.shape[0],train_df2.shape[1]))
print("The test dataset had %d rows and %d columns" %(test_df2.shape[0],test_df2.shape[1]))

The train dataset has 614 rows and 12 columns
The test dataset had 367 rows and 11 columns


In [93]:
#10. Fit a ridge regression model.
## Put the explanatory variables and the target variables in two different columns

from sklearn.linear_model import RidgeClassifier
x = train_df2[['ApplicantIncome', 'CoapplicantIncome', 'Credit_History',
       'Dependents', 'Education', 'Gender', 'LoanAmount',
       'Loan_Amount_Term','Married',
       'Property_Area', 'Self_Employed']]
y=train_df2['Loan_Status']


model_RCF = RidgeClassifier().fit(x,y)


In [95]:
#11. Obtain predicted values for the ridge regression using test data.
# print(model_LRS.score(x,y))
# print(model_LRS.coef_)
# print(model_LRS.intercept_)
RCF_predicted = model_RCF.predict(test_df2)
RCF_predicted
RCF_submission = pd.DataFrame({"Loan_ID":Loan_ID1,"Loan_Status":RCF_predicted})
RCF_submission.Loan_Status = np.where(RCF_submission.Loan_Status == 1,"Yes","No")
RCF_submission
RCF_submission.to_csv("LoanPredictionIII/Ridge_Classifier_submission.csv")


In [96]:
#12. Fit a random forest model.
from sklearn.ensemble import RandomForestClassifier
x = train_df2[['ApplicantIncome', 'CoapplicantIncome', 'Credit_History',
       'Dependents', 'Education', 'Gender', 'LoanAmount',
       'Loan_Amount_Term','Married',
       'Property_Area', 'Self_Employed']]
y=train_df2['Loan_Status']


model_RFC = RandomForestClassifier().fit(x,y)


In [100]:
#13. Obtain predicted values for the random forest using test data.
#print(model_RFC.score(x,y))
RFC_predicted = model_RFC.predict(test_df2)
RFC_submission = pd.DataFrame({"Loan_ID":Loan_ID1,"Loan_Status":RFC_predicted})
RFC_submission.Loan_Status = np.where(RFC_submission.Loan_Status == 1,"Y","N")
RFC_submission
RFC_submission.to_csv("LoanPredictionIII/RandomForest_Classifier_submission.csv")


In [101]:
#14. Fit an Support Vector Classifier (SVC)
from sklearn.svm import SVC
#clf = SVC()

model_SVC =SVC(C=1, gamma=1e-06, kernel="linear",
           max_iter=-1, probability=False, random_state=None, shrinking=True,
           tol=0.001, verbose=False,cache_size=40).fit(x, y)
model_SVC

SVC(C=1, cache_size=40, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1e-06, kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [102]:
#15. Obtain predicted values for the (SVC) model using test data.
SVC_predicted = model_SVC.predict(test_df2)
SVC_submission = pd.DataFrame({"Loan_ID":Loan_ID1,"Loan_Status":SVC_predicted})
SVC_submission.Loan_Status = np.where(SVC_submission.Loan_Status == 1,"Y","N")
#SVC_submission
SVC_submission.to_csv("LoanPredictionIII/Support_Vector_Classifier_submission.csv")


In [103]:
train_df

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,dataset
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y,train
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N,train
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y,train
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y,train
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y,train
5,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y,train
6,Male,Yes,0,Not Graduate,No,2333,1516.0,95.0,360.0,1.0,Urban,Y,train
7,Male,Yes,3+,Graduate,No,3036,2504.0,158.0,360.0,0.0,Semiurban,N,train
8,Male,Yes,2,Graduate,No,4006,1526.0,168.0,360.0,1.0,Urban,Y,train
9,Male,Yes,1,Graduate,No,12841,10968.0,349.0,360.0,1.0,Semiurban,N,train
